In [2]:
from pathlib import Path
from pprint import pprint

def list_daily_flow_files(base_dir: Path | None = None):
    """
    1. 取得 '每日各站進出站人數' 資料夾
    2. 取得其中所有 CSV 檔案的絕對路徑
    3. 排除 manifest.csv 與 schema.csv
    """
    if base_dir is None:
        base_dir = Path.cwd()  # 與 lesson19_2.ipynb 做法一致
    target_dir = base_dir / "每日各站進出站人數"
    if not target_dir.is_dir():
        raise FileNotFoundError(f"找不到資料夾: {target_dir}")

    excluded = {"manifest.csv", "schema.csv"}
    csv_paths = sorted(
        p.resolve()
        for p in target_dir.glob("*.csv")
        if p.name not in excluded
    )
    return target_dir.resolve(), csv_paths

folder_abs_path, file_abs_paths = list_daily_flow_files()

print("資料夾絕對路徑:")
print(folder_abs_path)
print("\n檔案絕對路徑 (已排除 manifest.csv, schema.csv):")
file_abs_paths = [str(p) for p in file_abs_paths]
print(file_abs_paths)
pprint([str(p) for p in file_abs_paths])

資料夾絕對路徑:
C:\Users\user\OneDrive\文件\Github\20250625_Gjun_Python-AI\0827_lesson19\每日各站進出站人數

檔案絕對路徑 (已排除 manifest.csv, schema.csv):
[]
[]


In [3]:
import pandas as pd
current_dir = Path.cwd()
# 若 CSV 與此 notebook 同資料夾
csv_path = current_dir / "台鐵車站資訊.csv"
stations_df = pd.read_csv(csv_path)
#display(stations_df.head())
stations_df = stations_df.reindex(columns=["stationCode", "stationName"])

#欄位名稱更改為[車站代碼, 車站名稱]
stations_df.columns = ["車站代碼", "車站名稱"]
stations_df

,車站代碼,車站名稱
0,900,基隆
1,910,三坑
2,920,八堵
3,930,七堵
4,940,百福
...,...,...
238,7360,瑞芳
239,7361,海科館
240,7362,八斗子
241,7380,四腳亭


In [4]:
#建立一個function
#要concate下面迴圈的所有merged_df
def process_yearly_data(file_abs_paths, stations_df):
    merged_dfs = []
    for csv_path in file_abs_paths:
        year_df = pd.read_csv(csv_path)
        year_df.columns = ["日期", "車站代碼", "進站人數", "出站人數"]
        #display(year_df.head())
        #日期欄位目前是int64, 需要轉換為datetime格式
        year_df["日期"] = pd.to_datetime(year_df["日期"], format="%Y%m%d")
        merged_df = pd.merge(year_df, stations_df, on="車站代碼")
        merged_df = merged_df.reindex(columns=["日期","車站名稱","進站人數","出站人數"])
        merged_df.head()
        #將欄位:日期,變為index
        merged_df.set_index("日期", inplace=True)
        merged_dfs.append(merged_df)
    return pd.concat(merged_dfs)
result_df = process_yearly_data(file_abs_paths, stations_df)
result_df

ValueError: No objects to concatenate